In [409]:
from pysal.contrib.spint.gravity import  BaseGravity, Gravity, Production, Attraction, Doubly
import numpy as np
import pysal as ps
import pandas as pd

In [410]:
#Load NYC bike data - trips between census tract centroids
bikes = pd.read_csv(ps.examples.get_path('nyc_bikes_ct.csv'))
bikes.head()

,Unnamed: 0,index,count,d_cap,d_tract,distance,end station latitude,end station longitude,o_cap,o_tract,...,weighted,total_out,total_in,o_hub,d_hub,od_hub,SX,SY,EX,EY
0,0,0,5709,255.0,600,NaN,40.712899,-73.989865,162.0,202,...,0.0,56352,69165,hub,hub,hub,585995.353038,4.507417e+06,585322.159723,4.507378e+06
1,1,1,4010,595.0,600,NaN,40.712899,-73.989865,774.0,700,...,0.0,160040,69165,hub,hub,hub,583785.918305,4.506573e+06,585322.159723,4.507378e+06
2,2,2,1906,170.0,600,NaN,40.712899,-73.989865,141.0,800,...,0.0,34254,69165,hub,hub,non_hub,585018.109713,4.507320e+06,585322.159723,4.507378e+06
3,3,3,1192,255.0,600,NaN,40.712899,-73.989865,291.0,900,...,0.0,46446,69165,hub,hub,non_hub,583444.520998,4.506199e+06,585322.159723,4.507378e+06
4,4,4,484,85.0,600,NaN,40.712899,-73.989865,57.0,1002,...,0.0,15916,69165,hub,hub,non_hub,586462.456350,4.507937e+06,585322.159723,4.507378e+06


In [411]:
#Process data

#Remove intrazonal flows
bikes = bikes[bikes['o_tract'] != bikes['d_tract']]

#Set zero attirbute values to a small constant
bikes.ix[bikes.o_sq_foot == 0, 'o_sq_foot'] = 1
bikes.ix[bikes.d_sq_foot == 0, 'd_sq_foot'] = 1
bikes.ix[bikes.o_cap == 0, 'o_cap'] = 1
bikes.ix[bikes.d_cap == 0, 'd_cap'] = 1
bikes.ix[bikes.o_housing == 0, 'o_housing'] = 1
bikes.ix[bikes.d_housing == 0, 'd_housing'] = 1

#Flows between tracts
flows = bikes['count'].values.reshape((-1,1))

#Origin variables: square footage of buildings, housing units, total station capacity
o_vars = np.hstack([bikes['o_sq_foot'].values.reshape((-1,1)),
                    bikes['o_housing'].values.reshape((-1,1)),
                    bikes['o_cap'].values.reshape((-1,1))])

#Destination variables: square footage of buildings, housing units, total station capacity
d_vars = np.hstack([bikes['d_sq_foot'].values.reshape((-1,1)),
                    bikes['d_housing'].values.reshape((-1,1)),
                    bikes['d_cap'].values.reshape((-1,1))])

#Trip "cost" in time (seconds)
cost = bikes['tripduration'].values.reshape((-1,1))

#Origin ids
o = bikes['o_tract'].astype(str).values.reshape((-1,1))

#destination ids
d = bikes['d_tract'].astype(str).values.reshape((-1,1))

print len(bikes), ' OD pairs between census tracts after filtering out intrazonal flows'

14042  OD pairs between census tracts after filtering out intrazonal flows


In [412]:
#Estimate gravity model using exponential function of distance-decay

grav= Gravity(flows, o_vars, d_vars, cost, 'exp')

print grav.params

[ 0.09898099  0.05748786  0.50319944  0.06920194  0.06408526  0.39371417
 -0.00226671]


In [413]:
#predicted response values using covariates
yhat = (0.09898099*np.log(o_vars[:,0])) + (0.05748786*np.log(o_vars[:,1])) + (0.50319944*np.log(o_vars[:,2])) + (0.06920194*np.log(d_vars[:,0])) + (0.06408526 *np.log(d_vars[:,1])) + (0.39371417*np.log(d_vars[:,2])) + (-0.00226671*cost[:,0]) 

In [414]:
#predicted response values using covariates and adding +1 to distance
yhat2 = (0.09898099*np.log(o_vars[:,0])) + (0.05748786*np.log(o_vars[:,1])) + (0.50319944*np.log(o_vars[:,2])) + (0.06920194*np.log(d_vars[:,0])) + (0.06408526 *np.log(d_vars[:,1])) + (0.39371417*np.log(d_vars[:,2])) + (-0.00226671*(cost[:,0]+1.0)) 

In [415]:
#predicted response variable before exponential transofrmation to satisfy log link
print yhat[0]
print np.exp(yhat[0])

7.63611161081
2071.67266526


In [416]:
#predicted response variable for 1 unit increase in distance before exponential transofrmation to satisfy log link
print yhat2[0]
print np.exp(yhat2[0])

7.63384490081
2066.9821022


In [417]:
#Difference in predicted response variables (+1 distance) before and after exponenetiation
print 'The difference in responses,', yhat[0] - yhat2[0], ', times 100 is the percentage (relative) change'
print 'The difference in responses,', np.exp(yhat[0]) - np.exp(yhat2[0]), ', is the absolute change'

The difference in responses, 0.00226671 , times 100 is the percentage (relative) change
The difference in responses, 4.69056306565 , is the absolute change


In [418]:
#Similarly, we can look at the % change from a 1 unit increase in distance
print ((np.exp(yhat[0]) - np.exp(yhat2[0]))/ np.exp(yhat[0]))*100
print np.exp(yhat[0])*0.00226414295284

0.226414295284
4.69056306565


In [419]:
#Here the % change is the same as (1 - np.exp(beta))*100
print (1.0 - np.exp(-0.00226671))*100

0.226414295284


In [420]:
#Which is why we can compute the effect of increasing distance +1
#w/o using the entire linear predictor
print 2071.67266526*np.exp(-0.00226671)
print '2071.67266526 - 2066.98210219 = ', 2071.67266526 - 2066.98210219
print 2071.67266526 - 2066.9821021943644, 'absolute difference, which is the same .2264% difference'

2066.98210219
2071.67266526 - 2066.98210219 =  4.69056307
4.69056306564 absolute difference, which is the same .2264% difference


In [421]:
#This relative change is the same for all observations
print yhat-yhat2

[ 0.00226671  0.00226671  0.00226671 ...,  0.00226671  0.00226671
  0.00226671]


In [422]:
#Though the absolute difference is unique for each observation
print np.exp(yhat)-np.exp(yhat2)

[ 4.69056307  9.74483347  4.58169324 ...,  0.42161474  0.74276242
  3.88429576]


In [423]:
#Estimate gravity model using power function of distance-decay

grav= Gravity(flows, o_vars, d_vars, cost, 'pow')

print grav.params

[ 0.28859837  0.15098447  0.46334398  0.26708303  0.1638457   0.36553998
 -1.54771364]


In [424]:
#predicted response values using covariates
yhat = (0.28859837*np.log(o_vars[:,0])) + (0.15098447*np.log(o_vars[:,1])) + (0.46334398*np.log(o_vars[:,2])) + (0.26708303*np.log(d_vars[:,0])) + (0.1638457 *np.log(d_vars[:,1])) + (0.36553998*np.log(d_vars[:,2])) + (-1.54771364*np.log(cost[:,0])) 

In [425]:
#predicted response values using covariates and adding +1 to distance
yhat2 = (0.28859837*np.log(o_vars[:,0])) + (0.15098447*np.log(o_vars[:,1])) + (0.46334398*np.log(o_vars[:,2])) + (0.26708303*np.log(d_vars[:,0])) + (0.1638457 *np.log(d_vars[:,1])) + (0.36553998*np.log(d_vars[:,2])) + (-1.54771364*np.log(cost[:,0]+1.0)) 

In [426]:
#predicted response variable before exponential transofrmation to satisfy log link
print yhat[0]
print np.exp(yhat[0])

7.26071186285
1423.26934924


In [427]:
#predicted response variable for 1 unit increase in distance before exponential transofrmation to satisfy log link
print yhat2[0]
print np.exp(yhat2[0])

7.25745127664
1418.6362143


In [428]:
#Difference in predicted response variables (+1 distance) before and after exponenetiation
print 'The difference in responses,', yhat[0] - yhat2[0], ', times 100 is the (relative) percentage change'
print 'The difference in responses,', np.exp(yhat[0]) - np.exp(yhat2[0]), ', is the absolute change'

The difference in responses, 0.00326058621352 , times 100 is the (relative) percentage change
The difference in responses, 4.63313494557 , is the absolute change


In [429]:
#Similarly, we can look at the % change from a 1 unit increase in distance
print ((np.exp(yhat[0]) - np.exp(yhat2[0]))/ np.exp(yhat[0]))*100
print np.exp(yhat[0])*0.00325527627503

0.325527627503
4.63313494557


In [430]:
#DIFFERENCE #1
#Here the % change is NOT the same as (1 - np.exp(beta))*100 !!!!!
print (1.0 - np.exp(-1.54771364))*100
print 'The reason for this is because the coefficient represents a 1 unit change in log(distance) and not distance itself'

78.7266195716
The reason for this is because the coefficient represents a 1 unit change in log(distance) and not distance itself


In [431]:
#Which is why we CANNOT compute the effect of increasing distance +1
#w/o using the entire linear predictor in the same manner
print 1423.26934924*np.exp(-1.54771364)
print '1423.26934924 - 302.777503185 = ', 1423.26934924 - 302.777503185
print '1120.49184606 != 4.63313494'
print '1120.49184606 absolute difference, which is 78.7% difference and not the same as .325% differemce'

302.777503185
1423.26934924 - 302.777503185 =  1120.49184606
1120.49184606 != 4.63313494
1120.49184606 absolute difference, which is 78.7% difference and not the same as .325% differemce


In [432]:
#What we are actually computing is the effect of increasing distance from 474.17360501 to 1288.9374940336004 (+814.7638890236003)
print np.log(474.17360501)
print '6.16157350994 + 1 =', np.log(474.17360501) + 1
print 'exp(7.16157350994) = ', np.exp(7.16157350994)
print 'If we plug in 7.16157350994 instead of log(474.17360501) into our linear predictor we get 302.77750318869766'

6.16157350994
6.16157350994 + 1 = 7.16157350994
exp(7.16157350994) =  1288.93749404
If we plug in 7.16157350994 instead of log(474.17360501) into our linear predictor we get 302.77750318869766


In [433]:
#Which leads to DIFFERENCE #2
#The relative change is NOT the same for all observations
print yhat-yhat2

[ 0.00326059  0.0020194   0.003905   ...,  0.0010131   0.00111467
  0.00264439]


In [434]:
#And the absolute difference is still unique for each observation
print np.exp(yhat)-np.exp(yhat2)

[ 4.63313495  8.44571954  6.42875335 ...,  0.32398569  0.44679361
  2.8630947 ]


In [435]:
#Instead, if we want to compute the change from 1 absolute unit without the linear predictor
#we need to normalize the coefficient to represent the change associated with 1 distance unit (not 1 log(distance))
#Due to difference #2, this normaliation will be unique for each observation
#And is computed as delta_unit = log(cost+1) - log(cost) 

print 'In this case log(475.17360501) - log(474.17360501) =', np.log(475.17360501) - np.log(474.17360501)

#Then we multiply this delta by the distance coefficient in order to make the formula work for a single unit
print 'Then 1423.26934924 * exp(-1.5477*0.00210671155779) =', 1423.26934924 * np.exp(-1.5477*0.00210671155779)
print 'Which is the same change we predict for a 1 unit change using our linear predictor'
print 'The original predicted response (1423.26934924) and the delta (0.00210671155779) can be substituted'
print 'for vectors in order to obtain results for every observation, which is needed due to difference #2'

In this case log(475.17360501) - log(474.17360501) = 0.00210671155779
Then 1423.26934924 * exp(-1.5477*0.00210671155779) = 1418.63625506
Which is the same change we predict for a 1 unit change using our linear predictor
The original predicted response (1423.26934924) and the delta (0.00210671155779) can be substituted
for vectors in order to obtain results for every observation, which is needed due to difference #2


In [436]:
#A final note, is that if you want to interpret any of your other covariates pertaining to origins or destinations
#in either the exponential or power distance decay specifications in their actual units and not log(units) then you
#need to apply this same normalization as all of the origin/destination covariates are automaticallt logged in
#order to obtain the correct Poisson GLM spatial interaction model specification